In [16]:
from huggingface_hub import login
login("...")

### Load SFT framework

In [1]:
!git clone https://github.com/turbo-llm/turbo-alignment.git

Cloning into 'turbo-alignment'...
remote: Enumerating objects: 4223, done.
remote: Counting objects: 100% (1144/1144), done.
remote: Compressing objects: 100% (363/363), done.
remote: Total 4223 (delta 831), reused 1048 (delta 774), pack-reused 3079 (from 1)
Receiving objects: 100% (4223/4223), 58.33 MiB | 13.75 MiB/s, done.
Resolving deltas: 100% (2791/2791), done.


In [2]:
%cd turbo-alignment

/content/turbo-alignment


### Load model from HF

In [18]:
!git lfs install
!git lfs clone https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
#!git lfs clone https://huggingface.co/Vikhrmodels/Vikhr-Llama-3.2-1B-Instruct

Updated git hooks.
Git LFS initialized.
          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'Llama-3.2-3B-Instruct'...
fatal: could not read Username for 'https://huggingface.co': No such device or address
Error(s) during clone:
git clone failed: exit status 128


### Test generation on base model and check architecture

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

def load_model_and_tokenizer(model_name, use_auth_token=False):
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    return model, tokenizer

def generate_text(model,
                  tokenizer,
                  prompt,
                  temperature=1.0,
                  max_length=100):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=temperature
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [9]:
model_name = "Vikhr-Gemma-2B-instruct"
model, tokenizer = load_model_and_tokenizer(model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps

In [11]:
prompt = "Напиши рекламу для Атом.Око"
generated_text_rugpt3l = generate_text(model,
                                       tokenizer,
                                       prompt)

print(f"{generated_text_rugpt3l}")

Напиши рекламу для Атом.Оконов


In [22]:
clear_memory()

del model
del tokenizer

torch.cuda.synchronize()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

### Set parameters and disable w&b

In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Install dependencies

In [3]:
%pip install poetry
!poetry install

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: turbo-alignment (0.0.4)Installing the current project: turbo-alignment (0.0.4)


In [ ]:
!poetry update

In [ ]:
# Special for Google Colab
%pip install -q peft==0.9 cached-path pydantic-settings loguru triton pytorchvideo datasets evaluate rouge_score clearml timm wandb vllm

### Test base model

In [21]:
%%time
!python -m turbo_alignment inference_chat --inference_settings_path inference_test_setting.json

2024-11-26 12:13:27.632323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-26 12:13:27.653416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-26 12:13:27.659608: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 12:13:27.674939: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 12:13:28.879853: W tensorflow/comp

### Fine-tuning model

In [23]:
!nvidia-smi

Tue Nov 26 10:57:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0              27W /  70W |  10273MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
!python -m turbo_alignment train_sft --experiment_settings_path lora_sft.json

2024-11-26 10:53:42.138805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-26 10:53:42.176108: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-26 10:53:42.185979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 10:53:44.304014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_settings" in BaseTrainExperimentSettings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting

In [ ]:
# Clear training logs
%rm -r log_path/*

### Test model with adapter

In [ ]:
%%time
!python -m turbo_alignment inference_chat --inference_settings_path lora_inference_test_setting.json

2024-11-25 21:06:43.442998: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-25 21:06:43.462883: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-25 21:06:43.468861: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 21:06:44.663446: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_settings" in BaseTrainExperimentSettings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting